Preliminary work:
    - count 0s, 0.1s, and missing values

In [1]:
import synapseclient
import pandas as pd
import numpy as np
import json

syn = synapseclient.Synapse()
syn.login(silent=True)

overall_scores = 'syn25905783'
overall_scores = syn.get(overall_scores, version=57)
overall_scores = pd.read_json(overall_scores.path, orient='records')

overall_scores.columns

Index(['ENSG', 'GeneName', 'Logsdon', 'GeneticsScore', 'OmicsScore',
       'LiteratureScore'],
      dtype='object')

min and max of each field as column:

In [2]:
neo_matrix = {}
select_columns = overall_scores.columns[2:]
for col in select_columns:
    neo_matrix[col] = {}
    neo_matrix[col]["distribution"] = list(pd.cut(overall_scores[col], bins=10, precision=3).value_counts())
    neo_matrix[col]["min"] = np.around(overall_scores[col].min(), 3)
    neo_matrix[col]["max"] = np.around(overall_scores[col].max(), 3)
    neo_matrix[col]["mean"] = np.around(overall_scores[col].mean(), 3)
    discard, neo_matrix[col]["bins"] = list(pd.cut(overall_scores[col], bins=10, precision=3, retbins=True))
    neo_matrix[col]["bins"] = np.around(neo_matrix[col]["bins"].tolist()[1:], 3)
    base = [0, *neo_matrix[col]["bins"][:-1]]
    neo_matrix[col]["bins"] = zip(base, neo_matrix[col]["bins"])
    neo_matrix[col]["bins"] = list(neo_matrix[col]["bins"])
    neo_matrix[col]["first_quartile"] = np.around(overall_scores[col].quantile(q=0.25, interpolation='midpoint'))
    neo_matrix[col]["third_quartile"] = np.around(overall_scores[col].quantile(q=0.75, interpolation='midpoint'))

In [3]:
additional_data = [{'name': 'Overall Score', 'syn_id': 'syn25913473', 'wiki_id': '613107'},
                    {'name': 'Genetics Score', 'syn_id': 'syn25913473', 'wiki_id': '613104'},
                    {'name': 'Omics Score', 'syn_id': 'syn25913473', 'wiki_id': '613106'},
                    {'name': 'Omics Score', 'syn_id': 'syn25913473', 'wiki_id': '613105'}
                  ]
for col, additional in zip(neo_matrix.keys(), additional_data):
    neo_matrix[col]['name'] = additional['name']
    neo_matrix[col]['syn_id'] = additional['syn_id']
    neo_matrix[col]['wiki_id'] = additional['wiki_id']
    
print(neo_matrix)


{'Logsdon': {'distribution': [10961, 4424, 3915, 3383, 2638, 1722, 964, 611, 203, 32], 'min': 0.0, 'max': 5.978, 'mean': 1.369, 'bins': [(0, 0.598), (0.598, 1.196), (1.196, 1.794), (1.794, 2.391), (2.391, 2.989), (2.989, 3.587), (3.587, 4.185), (4.185, 4.783), (4.783, 5.381), (5.381, 5.978)], 'first_quartile': 0.0, 'third_quartile': 2.0, 'name': 'Overall Score', 'syn_id': 'syn25913473', 'wiki_id': '613107'}, 'GeneticsScore': {'distribution': [8884, 5123, 3896, 3883, 3618, 1727, 472, 86, 13, 2], 'min': 0.022, 'max': 3.0, 'mean': 0.751, 'bins': [(0, 0.32), (0.32, 0.618), (0.618, 0.915), (0.915, 1.213), (1.213, 1.511), (1.511, 1.809), (1.809, 2.107), (2.107, 2.404), (2.404, 2.702), (2.702, 3.0)], 'first_quartile': 0.0, 'third_quartile': 1.0, 'name': 'Genetics Score', 'syn_id': 'syn25913473', 'wiki_id': '613104'}, 'OmicsScore': {'distribution': [8333, 2068, 1304, 1206, 1084, 887, 758, 553, 447, 392], 'min': 0.0, 'max': 2.0, 'mean': 0.592, 'bins': [(0, 0.2), (0.2, 0.4), (0.4, 0.6), (0.6, 0.

In [4]:
with open('../output/distribution_data.json', 'w') as outfile:
    json.dump(neo_matrix, outfile)